## Import

In [3]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore') 

In [21]:
import lightgbm as lgb

## Fixed Random-Seed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data

In [93]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [94]:
building = pd.read_csv('./data/building_info.csv')

## Train Data Pre-Processing

In [95]:
def train_test_split(df, th):
    train = df[df['일시'].str[:8].astype(int) < th].reset_index(drop=True)
    test = df[df['일시'].str[:8].astype(int) >= th].reset_index(drop=True)
    return train, test

In [96]:
def preprocess_x(df):
    df = df.fillna(0)
    #시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
    df['month'] = df['일시'].apply(lambda x : int(x[4:6]))
    df['day'] = df['일시'].apply(lambda x : int(x[6:8]))
    df['time'] = df['일시'].apply(lambda x : int(x[9:11]))
    df = df.merge(building.iloc[:, :4])
    df['건물유형'] = df['건물유형'].astype('category').cat.codes
#     df = df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
    df = df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
    return df

In [97]:
train_df, test_df = train_test_split(train_df, 20220820)

In [98]:
train_x = preprocess_x(train_df)
train_y = train_df['전력소비량(kWh)']

In [99]:
test_x = preprocess_x(test_df)
test_y = test_df['전력소비량(kWh)']

## Regression Model Fit

In [13]:
model_rf = RandomForestRegressor()
model_rf.fit(train_x, train_y)

RandomForestRegressor()

In [129]:
model_lgb = lgb.LGBMRegressor(objective='regression', n_estimators=300)

In [130]:
# model_lgb = lgb.LGBMRegressor(objective='regression')
model_lgb.fit(train_x, train_y)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001003 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1000
[LightGBM] [Info] Number of data points in the train set: 192000, number of used features: 11
[LightGBM] [Info] Start training from score 2448.539357


LGBMRegressor(n_estimators=300, objective='regression')

## Inference

In [102]:
def SMAPE(y, pred):
    smape = abs((y - pred))/((abs(y) + abs(pred)) / 2) * 100
    smape = np.mean(smape)
    return smape

def mae(y, pred):
    return np.mean(abs(y-pred))

In [131]:
pred = model_lgb.predict(test_x)

In [132]:
SMAPE(test_y, pred)

15.798359023528862

In [133]:
mae(test_y, pred)

352.1086433302414

In [71]:
eda = test_x.copy()

In [73]:
eda['pred'], eda['y'] = pred, test_y

In [75]:
eda['gap'] = eda['y'] - eda['pred']

In [77]:
eda.sort_values('gap')

,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time,건물유형,연면적(m2),냉방면적(m2),pred,y,gap
3155,29.6,0.0,2.7,61.0,8,21,11,2,578484.113,501381.53,23785.846815,15517.60,-8268.246815
3154,29.1,0.0,3.2,61.0,8,21,10,2,578484.113,501381.53,23437.946398,15260.00,-8177.946398
3156,30.5,0.0,2.7,59.0,8,21,12,2,578484.113,501381.53,23513.283952,15996.40,-7516.883952
3157,31.5,0.0,1.7,59.0,8,21,13,2,578484.113,501381.53,23848.714994,16587.20,-7261.514994
3158,31.6,0.0,1.6,54.0,8,21,14,2,578484.113,501381.53,23853.408656,16864.40,-6989.008656
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,31.7,0.0,2.3,64.0,8,20,14,0,205884.000,150000.00,2948.549114,7550.64,4602.090886
518,31.3,0.0,1.5,63.0,8,21,14,0,205884.000,150000.00,3029.387746,7642.08,4612.692254
519,31.2,0.0,2.3,61.0,8,21,15,0,205884.000,150000.00,2992.269004,7608.96,4616.690996
517,30.4,0.0,1.3,66.0,8,21,13,0,205884.000,150000.00,3051.354047,7683.84,4632.485953


### 건물별 모델

In [134]:
models = {}

In [135]:
from tqdm import tqdm

In [151]:
for i in tqdm(range(1, 101)):
    _x = train_x[train_x['건물번호'] == i]
    _x = _x.drop(columns=['건물번호', '건물유형', '연면적(m2)', '냉방면적(m2)'])
    _y = train_y[_x.index]
    model_lgb = lgb.LGBMRegressor(objective='regression', n_estimators=300)
    model_lgb.fit(_x, _y)
    models[i] = model_lgb

  2%|█▉                                                                                               | 2/100 [00:00<00:08, 11.96it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 2643.372748
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 1676.138344
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000101 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Sta

  4%|███▉                                                                                             | 4/100 [00:00<00:08, 11.94it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 993.510891
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 2725.884560
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000076 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 414
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Star

  6%|█████▊                                                                                           | 6/100 [00:00<00:08, 11.49it/s]

...

 96%|████████████████████████████████████████████████████████████████████████████████████████████▏   | 96/100 [00:08<00:00, 10.74it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000100 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 2801.350094
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000101 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 1227.598031
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 381
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Sta

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.71it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 406
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 1160.090812
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000092 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 397
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 7
[LightGBM] [Info] Start training from score 821.566251


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.71it/s]


In [154]:
preds = []

In [155]:
for i in tqdm(range(1, 101)):
    _x = test_x[test_x['건물번호'] == i]
    _x = _x.drop(columns=['건물번호', '건물유형', '연면적(m2)', '냉방면적(m2)'])
    pred = models[i].predict(_x).tolist()
    preds.extend(pred)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 671.63it/s]


In [156]:
preds = np.array(preds)

In [157]:
SMAPE(test_y, preds)

14.462616029933997

In [158]:
mae(test_y, preds)

351.1786298329664

In [159]:
train_x

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time,건물유형,연면적(m2),냉방면적(m2)
0,1,18.6,0.0,0.9,42.0,6,1,0,0,110634.00,39570.00
1,1,18.0,0.0,1.1,45.0,6,1,1,0,110634.00,39570.00
2,1,17.7,0.0,1.5,45.0,6,1,2,0,110634.00,39570.00
3,1,16.7,0.0,1.4,48.0,6,1,3,0,110634.00,39570.00
4,1,18.4,0.0,2.8,43.0,6,1,4,0,110634.00,39570.00
...,...,...,...,...,...,...,...,...,...,...,...
191995,100,28.1,0.0,1.9,85.0,8,19,19,11,57497.84,40035.23
191996,100,27.7,0.0,1.8,87.0,8,19,20,11,57497.84,40035.23
191997,100,27.7,0.0,1.4,87.0,8,19,21,11,57497.84,40035.23
191998,100,27.7,0.0,2.4,86.0,8,19,22,11,57497.84,40035.23


## Validate

In [30]:
preds

array([2076.1152, 2014.896 , 1882.224 , ...,  692.712 ,  673.4592,
        566.616 ])

In [31]:
test_y.values

array([1921.44, 1851.36, 1731.84, ...,  825.12,  640.08,  540.24])

## Submission

In [11]:
submission = pd.read_csv('./sample_submission.csv')
submission

,num_date_time,answer
0,1_20220825 00,0
1,1_20220825 01,0
2,1_20220825 02,0
3,1_20220825 03,0
4,1_20220825 04,0
...,...,...
16795,100_20220831 19,0
16796,100_20220831 20,0
16797,100_20220831 21,0
16798,100_20220831 22,0


In [12]:
submission['answer'] = preds
submission

,num_date_time,answer
0,1_20220825 00,2127.2688
1,1_20220825 01,2090.5008
2,1_20220825 02,2009.9712
3,1_20220825 03,1981.9440
4,1_20220825 04,1946.7744
...,...,...
16795,100_20220831 19,893.0712
16796,100_20220831 20,784.7448
16797,100_20220831 21,748.2216
16798,100_20220831 22,654.1728


In [13]:
submission.to_csv('./baseline_submission.csv', index=False)